Importing CSV to process dataset. 

In [29]:
import csv

Creating a dictionary to store corresponding coloumns and missing data. And code to print out how much data is missing for each of the required coloumns. 

In [ ]:
missing_data = {}
required_headers = ["P_MASS", "P_RADIUS", "P_DENSITY", "P_GRAVITY", "P_ESCAPE", "P_TYPE", "P_PERIOD", "P_SEMI_MAJOR_AXIS", "P_ECCENTRICITY", 
                    "P_INCLINATION", "P_OMEGA", "P_PERIASTRON", "P_APASTRON", "P_IMPACT_PARAMETER", "P_HILL_SPHERE", "S_MASS", "S_RADIUS",
                    "S_LUMINOSITY", "S_TEMPERATURE", "S_AGE", "S_METALLICITY", "S_LOG_G", "S_TYPE", "S_MAG", "S_DISC", "S_MAGNETIC_FIELD",
                    "S_SNOW_LINE", "S_TIDAL_LOCK", "P_DETECTION", "P_DISTANCE"]

with open("..\\datasets\\full_data.csv", "r") as file:
    content = csv.reader(file)
    headers = next(content) 
    for row in content:
        for col in range(112):
            if row[col] == "":
                if headers[col] in required_headers:
                    if headers[col] not in missing_data:
                        missing_data[headers[col]] = 0
                    missing_data[headers[col]] += 1



for name in headers:
    if name not in missing_data:
        missing_data[name] = 0
print("PROPERTY:NOS. OF MISSING DATA")
for key, value in missing_data.items():
    print(f"{key}:{value}")


PROPERTY:NOS. OF MISSING DATA
P_RADIUS:909
P_INCLINATION:3204
P_IMPACT_PARAMETER:2639
S_AGE:2017
S_DISC:4048
S_MAGNETIC_FIELD:4048
P_ESCAPE:3342
P_GRAVITY:3342
P_DENSITY:3342
P_OMEGA:3302
P_PERIOD:110
P_ECCENTRICITY:2668
S_MAG:179
S_METALLICITY:1206
S_RADIUS:325
S_LOG_G:473
S_LUMINOSITY:262
S_SNOW_LINE:262
S_TEMPERATURE:207
S_TYPE:2678
P_SEMI_MAJOR_AXIS:1681
P_MASS:2450
P_HILL_SPHERE:2502
P_TYPE:17
S_MASS:765
S_TIDAL_LOCK:765
P_DISTANCE:70
P_PERIASTRON:70
P_APASTRON:70
P_NAME:0
P_STATUS:0
P_MASS_ERROR_MIN:0
P_MASS_ERROR_MAX:0
P_RADIUS_ERROR_MIN:0
P_RADIUS_ERROR_MAX:0
P_YEAR:0
P_UPDATED:0
P_PERIOD_ERROR_MIN:0
P_PERIOD_ERROR_MAX:0
P_SEMI_MAJOR_AXIS_ERROR_MIN:0
P_SEMI_MAJOR_AXIS_ERROR_MAX:0
P_ECCENTRICITY_ERROR_MIN:0
P_ECCENTRICITY_ERROR_MAX:0
P_INCLINATION_ERROR_MIN:0
P_INCLINATION_ERROR_MAX:0
P_OMEGA_ERROR_MIN:0
P_OMEGA_ERROR_MAX:0
P_TPERI:0
P_TPERI_ERROR_MIN:0
P_TPERI_ERROR_MAX:0
P_ANGULAR_DISTANCE:0
P_IMPACT_PARAMETER_ERROR_MIN:0
P_IMPACT_PARAMETER_ERROR_MAX:0
P_TEMP_MEASURED:0
P_GEO_

Pritning the maximum and minimum data missing column names. Easier to analyze. 

In [31]:
max_missing = missing_data["P_RADIUS"]
max_missing_header = ""
min_missing = missing_data["P_RADIUS"]
min_missing_header = ""

for name in required_headers:
    if missing_data[name] > max_missing:
        max_missing = missing_data[name]
        max_missing_header = name
    if missing_data[name] < min_missing:
        min_missing = missing_data[name]
        min_missing_header = name

print(f"\nPROPERTY WITH MAX MISSING DATA: {max_missing_header} WITH {max_missing} MISSING ENTRIES")
print(f"PROPERTY WITH MIN MISSING DATA: {min_missing_header} WITH {min_missing} MISSING ENTRIES")


PROPERTY WITH MAX MISSING DATA: S_DISC WITH 4048 MISSING ENTRIES
PROPERTY WITH MIN MISSING DATA: P_DETECTION WITH 0 MISSING ENTRIES
